In [1]:
# %% [markdown]
# # OpenBB + QuantLib to generate risk free

# %%
from openbb import obb # In charge of the collection of SOFR data we will use to calculate risk free.
import pandas as pd # Used to interact with the data as dataframes 
import math # Supports calculations of specific mathematical functions in the paper
import numpy as np


# %% [markdown]
# ## Supporting functions

# %%
def load_sofr_history(start:str, end:str):
    """
    Obtains the SOFR data from FRED on the defined start and end dates.
    start and end must be in format YYYY-MM-DD.
    """
    df = obb.economy.fred_series("SOFR", start_date=start, end_date=end).to_dataframe()
    s = df/100
    s = s['SOFR']
    return s

# %%
def historical_rigorous_rf(
    start:str,
    end:str,
    horizon_days:int,
    day_count=360,
):
    """
    Generates the exact r value for the call pricing formula in page 12 of the PDF.
    It grabs the daily annualised SOFR rates from FRED (which are simple rates, not compounding) and calculates the daily return based on the annualised number.
    Interest Accrued in one day = SOFR annualised rate * 1/ 360
    start: Starting date of the needed dates. String in the form 'YYYY-MM-DD'
    end: End date of the needed dates. String in the form 'YYYY-MM-DD'
    horizon_days: In the context of the paper, this would be the time to maturity for the option in days (T-t). Integer.
    day_count: 360 or 365 depending on what type of denominator we want to use for the simple interest calculation. Set at 360 as it is more commonly used.
    """
    sofr = load_sofr_history(start, end)

    # Ensure daily grid and forward-fill
    idx = pd.date_range(start, end, freq="D")
    sofr = sofr.reindex(idx).ffill()

    dt = 1.0 / day_count
    one_day_growth = 1.0 + sofr * dt

    # Growth over next horizon_days
    growth_N = one_day_growth.shift(-1).rolling(horizon_days).apply(np.prod, raw=True)

    # Discount factor (e^{-r(T-t)})
    df_N = 1.0 / growth_N

    # tau = T-t in years
    tau = horizon_days / day_count
    # Final annualised values to use in the pricing equation
    r_ann = -np.log(df_N) / tau

    # The output is a pandas series where the index is a date,
    # and the value representing the date is the literal value of r in the pricing formula on page 12 of the PDF.
    return r_ann.dropna().rename(f"rf_{horizon_days}d_continuous_SOFR")

# %%
rf_6d = historical_rigorous_rf(
    start = '2024-01-01',
    end='2024-06-30',
    horizon_days=6
)

print(rf_6d)




2024-01-06    0.053396
2024-01-07    0.053246
2024-01-08    0.053113
2024-01-09    0.053096
2024-01-10    0.053096
                ...   
2024-06-25    0.053179
2024-06-26    0.053229
2024-06-27    0.053263
2024-06-28    0.053296
2024-06-29    0.053329
Freq: D, Name: rf_6d_continuous_SOFR, Length: 176, dtype: float64


In [2]:
from openbb import obb
import pandas as pd

# Get option chains for a specific stock for a specific date

def get_option_chains(symbol: str):
    """
    Get the latest option chains for a specific stock.

    Parameters:
    symbol (str): The stock symbol to get option chains for.

    """
    # Get the option chains for the specified stock and date
    option_chains = obb.derivatives.options.chains(symbol=symbol).to_dataframe()

    # Convert the result to a DataFrame
    

    return option_chains

In [3]:
import numpy as np
from scipy.stats import lognorm
from scipy.optimize import minimize
from scipy.integrate import quad

def lognormal_pdf(s:float, a:float, b:float):
    """
    Probability density function of the Lognormal Distribution. Used in the calculation of the mixture distribution in page 12.
    s (float) : Value whose probability we want to calculate
    a (float) : Mean of log-price.
    b (float) : Standard Deviation of log-price
    """
    return lognorm.pdf(s, s=b, scale=np.exp(a))

def mixture_pdf(s:float, a1:float, b1:float, a2:float, b2:float, q:float):
    """
    Calculates the weighted pdf from two parametrised log-normal distributions. Used in page 12 as the final pdf for our prices.
    s (float) : Value whose probability we want to calculate
    a1 (float) : Mean of log-price for the first distribution.
    b1 (float) : Standard Deviation of log-price for the first distribution.
    a2 (float) : Mean of log-price for the second distribution.
    b2 (float) : Standard Deviation of log-price for the second distribution.
    q (float) : Weight assigned to the first distribution (the second will have weight 1-q).
    """
    return q * lognormal_pdf(s, a1, b1) + (1 - q) * lognormal_pdf(s, a2, b2)

def implied_mean(a1, b1, a2, b2, q):
    """
    Calculates the implied mean of the calculated mixed_distribution. Will be used in the minimisation problem.

    a1 (float) : Mean of log-price for the first distribution.
    b1 (float) : Standard Deviation of log-price for the first distribution.
    a2 (float) : Mean of log-price for the second distribution.
    b2 (float) : Standard Deviation of log-price for the second distribution.
    q (float) : Weight assigned to the first distribution (the second will have weight 1-q).
    """
    return q * np.exp(a1 + 0.5*b1**2) + (1-q) * np.exp(a2 + 0.5*b2**2)

def call_price_model(X, r, tau, a1, b1, a2, b2, q):
    """
    Call price formula, defined as the sum of all weighted possible payoffs from holding a call today using the mixed pdf.
    
    :param X: Strike Price of the call option.
    :param r: Risk-free rate (should always be the rate generated in the data sources).
    :param tau: Time to maturity in years.
    :param a1: Mean of log-price for the first distribution.
    :param b1: Standard Deviation of log-price for the first distribution.
    :param a2: Mean of log-price for the second distribution.
    :param b2: Standard Deviation of log-price for the second distribution.
    :param q: Weight assigned to the first distribution (the second will have weight 1-q).
    """
    integrand = lambda s: (s - X) * mixture_pdf(s, a1, b1, a2, b2, q)
    value, _ = quad(integrand, X, np.inf)
    return np.exp(-r * tau) * value

def put_price_model(X, r, tau, a1, b1, a2, b2, q):
    """
    Put price formula, defined as the sum of all weighted possible payoffs from holding a call today using the mixed pdf.
    
    :param X: Strike Price of the call option.
    :param r: Risk-free rate (should always be the rate generated in the data sources).
    :param tau: Time to maturity in years.
    :param a1: Mean of log-price for the first distribution.
    :param b1: Standard Deviation of log-price for the first distribution.
    :param a2: Mean of log-price for the second distribution.
    :param b2: Standard Deviation of log-price for the second distribution.
    :param q: Weight assigned to the first distribution (the second will have weight 1-q).
    """
    integrand = lambda s: (X - s) * mixture_pdf(s, a1, b1, a2, b2, q)
    val, _ = quad(integrand, 0, X)
    return np.exp(-r * tau) * val


def objective_from_chain(theta, chain, r):
    """
    Objective function we want to minimise to find the optimal vector theta = [a1, b1, a2, b2, q]

    :param theta: Vector containing the distribution parameters and weights [a1, b1, a2, b2, q].
    :param chain: Dataframe containing the option chain data we will use for the calculations
    :param r: Adjusted risk-free rate calculated for the defined timeframe
    """

    a1, b1, a2, b2, q = theta

    tau = chain['dte'].iloc[0] / 365
    S0  = chain['underlying_price'].iloc[0]
    S_forward = S0 * np.exp(r * tau)

    error = 0.0

    for _, row in chain.iterrows():
        X = row['strike']
        market_price = 0.5 * (row['bid'] + row['ask'])

        if row['option_type'].lower() == 'call':
            model_price = call_price_model(X, r, tau, a1, b1, a2, b2, q)
        else:
            model_price = put_price_model(X, r, tau, a1, b1, a2, b2, q)

        error += (model_price - market_price)**2

    mean_err = (implied_mean(a1, b1, a2, b2, q) - S_forward)**2

    return error + mean_err



In [10]:
rfs = historical_rigorous_rf(start='2025-01-01', end='2025-12-26', horizon_days=6)
r = rfs.iloc[-1]

In [ ]:
chain = get_option_chains('AAPL')

x0 = [np.log(chain['underlying_price'].iloc[0]), 0.2,
      np.log(chain['underlying_price'].iloc[0]), 0.5,
      0.5]

bounds = [
    (None, None), (1e-6, None),
    (None, None), (1e-6, None),
    (0, 1)
]

res = minimize(objective_from_chain, x0, args=(chain, r),
               method='L-BFGS-B', bounds=bounds)

a1, b1, a2, b2, q = res.x


In [ ]:
res.x